In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import col #lit

tablename = "dimfoiflowrequeststatus"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    # spark.sql(f"DROP TABLE IF EXISTS {tablename};")

    query = f"""
        SELECT
            requeststatusid,
            name,
            description,
            case when isactive = true then 't' else 'f' end as isactive,
            'FOIMOD' AS sourceoftruth
        FROM foi_mod.foirequeststatuses
        order by requeststatusid;
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
            "requeststatusid AS requeststatusid",
            "name AS name",
            "description AS description",
            "isactive AS isactive",
            "sourceoftruth AS sourceoftruth"
        )
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.requeststatusid = source.requeststatusid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(set = {
        "name": col("source.name"),
        "description": col("source.description"),
        "isactive": col("source.isactive")
    }).whenNotMatchedInsert(values = {
        "requeststatusid": col("source.requeststatusid"),
        "name": col("source.name"),
        "description": col("source.description"),
        "isactive": col("source.isactive"),
        "sourceoftruth": col("source.sourceoftruth")
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        raise Exception("notebook failed") from e